In [ ]:
Sure! Here's the optimized version of your code with some improvements and error handling:

python
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report

def load_data(file_path):
    df = pd.read_csv(file_path)
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df.set_index('Timestamp', inplace=True)
    return df

def prepare_data(data, sequence_length):
    X = data.iloc[:, :-1]
    y = data.iloc[:, -1]
    return X, y

def classify(predictions, threshold=0.5):
    return [1 if pred >= threshold else 0 for pred in predictions]

def main():
    sequence_length = 10
    file_path = "hv_days.csv"  # Specify the path to your CSV file
    data = load_data(file_path)
    X, y = prepare_data(data, sequence_length)

    # Scale the features and target variable
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=28)

    # Balance the data using SMOTE
    smote = SMOTE(random_state=42)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

    # Create TimeseriesGenerators for training and validation
    train_generator = TimeseriesGenerator(X_train_balanced, y_train_balanced, length=sequence_length, batch_size=1)
    validation_generator = TimeseriesGenerator(X_test, y_test, length=sequence_length, batch_size=1)

    # Define LSTM model
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(sequence_length, X_train_balanced.shape[1])))
    model.add(Dense(1))
    model.compile(optimizer='adadelta', loss='mse')

    # Train the model
    model.fit(train_generator, validation_data=validation_generator, epochs=10)

    # Predict using the validation generator
    y_pred = model.predict(validation_generator)

    # Inverse transform the scaled predictions and true values to original values
    y_pred_original = scaler_y.inverse_transform(y_pred.reshape(-1, 1)).flatten()
    y_test_original = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()

    # Convert predictions to classes using the threshold (0.5 in this case)
    y_pred_classes = classify(y_pred_original, threshold=0.5)

    # Compute and print the classification report
    report = classification_report(y_test_original, y_pred_classes)
    print("Classification Report:")
    print(report)

if __name__ == "__main__":
    main()
